<a href="https://colab.research.google.com/github/awaaat/Machine_learning-Deep_learning/blob/main/Basic_text_classification_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#We are going to be performing basic text classification with tensorflow


1. We Import the Needed Libraries

In [2]:
import os
import re
import shutil
import string
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras import layers
from tensorflow.keras import losses

In [ ]:
#Let us fetch the dataset to be used
"""This will be a binary classification. We will be using the moview reviews dataset to classify the reviews as either positive or negative"""

In [3]:
dataset_url = "https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz"
dataset = tf.keras.utils.get_file(fname = "aclImdb_v1", origin = dataset_url, untar = True, cache_dir=".", cache_subdir = " ")
dataset_dir = os.path.join(os.path.dirname(dataset), "aclImdb")

84125825/84125825 [==============================] - 10s 0us/step


In [4]:
os.listdir(dataset_dir)

['imdbEr.txt', 'imdb.vocab', 'train', 'test', 'README']

In [5]:
#Let us fetch the training directory
train_dir = os.path.join(dataset_dir, "train")
os.listdir(train_dir)

['urls_pos.txt',
 'urls_neg.txt',
 'unsupBow.feat',
 'urls_unsup.txt',
 'pos',
 'labeledBow.feat',
 'unsup',
 'neg']

The aclImdb/train/pos and aclImdb/train/neg directories contain many text files, each of which is a single movie review. Let's take a look at one of them.

In [6]:
#Let us locate one single file
sample_file = os.path.join(train_dir, "pos/1181_9.txt")
with open(sample_file) as f:
  print(f.read())

Rachel Griffiths writes and directs this award winning short film. A heartwarming story about coping with grief and cherishing the memory of those we've loved and lost. Although, only 15 minutes long, Griffiths manages to capture so much emotion and truth onto film in the short space of time. Bud Tingwell gives a touching performance as Will, a widower struggling to cope with his wife's death. Will is confronted by the harsh reality of loneliness and helplessness as he proceeds to take care of Ruth's pet cow, Tulip. The film displays the grief and responsibility one feels for those they have loved and lost. Good cinematography, great direction, and superbly acted. It will bring tears to all those who have lost a loved one, and survived.


In [8]:
#Let us now load the dataset using the keras text_dataset_from_directory
"""We start by removing additional folders from the set"""
remove_dir = os.path.join(train_dir, "unsup")
shutil.rmtree(remove_dir)

In [9]:
#Let us then create a validation set from our training set
batch_size = 32
seed = 42
raw_training_ds = tf.keras.utils.text_dataset_from_directory(train_dir, batch_size = batch_size, validation_split = 0.2, seed = seed, subset = "training")

Found 25000 files belonging to 2 classes.
Using 20000 files for training.


In [13]:
#Nice. We can print a few samples, say 5
for text_batch, label_batch in raw_training_ds.take(1):
  for i in range(5):
    print(f"Review: {text_batch.numpy()[i]}")
    print(f"Label: {label_batch.numpy()[i]}")

Review: b'Great movie - especially the music - Etta James - "At Last". This speaks volumes when you have finally found that special someone.'
Label: 0
Review: b"I am shocked. Shocked and dismayed that the 428 of you IMDB users who voted before me have not given this film a rating of higher than 7. 7?!?? - that's a C!. If I could give FOBH a 20, I'd gladly do it. This film ranks high atop the pantheon of modern comedy, alongside Half Baked and Mallrats, as one of the most hilarious films of all time. If you know _anything_ about rap music - YOU MUST SEE THIS!! If you know nothing about rap music - learn something!, and then see this! Comparisons to 'Spinal Tap' fail to appreciate the inspired genius of this unique film. If you liked Bob Roberts, you'll love this. Watch it and vote it a 10!"
Label: 1
Review: b'What a lovely heart warming television movie. The story tells of a little five year old girl who has lost her daddy and finds it impossible to cope. Her mother is also very distres

In [ ]:
"""Notice the reviews contain raw text (with punctuation and occasional HTML tags like <br/>).
We willl need to handle these in the following section""